In [118]:
import sys
import itertools
import numpy as np 
import pandas as pd
import re
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder
#from pandas_profiling import ProfileReport

In [2]:
df = pd.read_csv('tracks_features.csv')

In [3]:
df['artist_ids'].values[0]

"['2d0hyoQ5ynDBnkvAbJKORj']"

In [4]:
df['artist_ids'] = df['artist_ids'].apply(lambda x: re.findall(r"'([^']*)'", x))

In [5]:
df['artist_ids'].values[0]

['2d0hyoQ5ynDBnkvAbJKORj']

In [6]:
df['artist_ids'] = df['artist_ids'].apply(lambda x: "".join(x))

In [7]:
df['artist_ids'].values[0];

In [8]:
df1 = pd.read_csv('artist_genres.csv')

In [9]:
df1['ID'].values[0];

In [10]:
df1.drop('Unnamed: 0', axis = 1,inplace = True)

In [11]:
df3 = df.merge(df1, how="left",left_on = 'artist_ids', right_on = 'ID')

In [12]:
df3.shape

(1204025, 26)

In [13]:
df3.Genres.values[0][0]

'['

In [14]:
def explicit_converter(x):
    if x:
        return 1 
    else:
        return 0

In [16]:
def decade_function(x):
    if x in range(1900,1991):
        return '1900-1990'
    elif x in range(1991,1996):
        return '1991-1995'
    elif x in range(1996,2001):
        return '1996-2000'
    elif x in range(2001,2006):
        return '2001-2005'
    elif x == 2006:
        return '2006' 
    elif x == 2007:
        return '2007'
    elif x == 2008:
        return '2008'
    elif x == 2009:
        return '2009'
    elif x == 2010:
        return '2010'
    elif x == 2011:
        return '2011'
    elif x == 2012:
        return '2012'
    elif x == 2013:
        return '2013'
    elif x == 2014:
        return '2014'
    elif x == 2015:
        return '2015'
    elif x == 2016:
        return '2016'
    elif x == 2017:
        return '2017'
    elif x == 2018:
        return '2018'
    elif x == 2019:
        return '2019'
    elif x == 2020:
        return '2020'
    elif x == 2021:
        return '2021'
        

In [17]:
def wrangle(df3):
    df3['artists_new'] = df3['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))
    df3['artists_new2'] = df3['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
    df3['artists_final'] = np.where(df3['artists_new'].apply(lambda x: not x), df3['artists_new2'], df3['artists_new'])
    df3['artists_song'] = df3.apply(lambda row: row['artists_final'][0]+row['name'],axis = 1)
    df3.drop_duplicates('artists_song',inplace = True)
    cols_to_drop = ['id', 'album_id','artists', 'artist_ids', 'track_number', 'disc_number', 'release_date', 'ID', 'artists_new', 'artists_new2', 'artists_song', 'duration_ms']
    df3.drop(columns = cols_to_drop, inplace = True)
    df3['Genres'] = df3['Genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", str(x))]) 
    df3['explicit'] = df3['explicit'].apply(explicit_converter)
    df3.rename(columns={"Genres": "genres"}, inplace = True)
    df3['year'] = df3['year'].replace(0, np.nan)
    df3.dropna(subset = ['year'], inplace = True)
    return df3

In [18]:
wrangle(df3)

,name,album,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,genres,artists_final
0,Testify,The Battle Of Los Angeles,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
1,Guerrilla Radio,The Battle Of Los Angeles,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
2,Calm Like a Bomb,The Battle Of Los Angeles,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
3,Mic Check,The Battle Of Los Angeles,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
4,Sleep Now In the Fire,The Battle Of Los Angeles,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204020,Gospel of Juke,Notch - EP,0,0.264,0.966,5,-6.970,0,0.0672,0.00935,0.002240,0.3370,0.415,159.586,4.0,2014.0,[slovak_electronic],[FVLCRVM]
1204021,Prism Visions,Notch - EP,0,0.796,0.701,11,-6.602,0,0.0883,0.10400,0.644000,0.0749,0.781,121.980,4.0,2014.0,[slovak_electronic],[FVLCRVM]
1204022,Tokyo 360,Notch - EP,0,0.785,0.796,9,-5.960,0,0.0564,0.03040,0.918000,0.0664,0.467,121.996,4.0,2014.0,[slovak_electronic],[FVLCRVM]
1204023,Yummy!,Notch - EP,0,0.665,0.856,6,-6.788,0,0.0409,0.00007,0.776000,0.1170,0.227,124.986,4.0,2014.0,[slovak_electronic],[FVLCRVM]


In [19]:
df.shape

(1204025, 24)

In [20]:
df4 = df3[~df3['genres'].str.len().eq(0)]

In [21]:
df4.shape

(558575, 18)

In [127]:
df4.to_csv('spotify_clean_v3.csv')

In [23]:
float_cols = df4.dtypes[df4.dtypes == 'float64'].index.values


In [24]:
df4['age'] = df4['year'].apply(np.int64)

<ipython-input-24-bf52b0787dc0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['age'] = df4['year'].apply(np.int64)


In [25]:
df4['age'] = df4['age'].apply(decade_function)

<ipython-input-25-115a0c2e1805>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['age'] = df4['age'].apply(decade_function)


In [26]:
ohe_enc = fe_OneHotEncoder(
    top_categories=None,
    variables=['age'], drop_last=True)  # to return k-1, false to return k


ohe_enc.fit(df4.fillna('Missing'))

OneHotEncoder(drop_last=True, variables=['age'])

In [27]:
df4 = ohe_enc.transform(df4.fillna('Missing'))

In [28]:
df4.shape

(558575, 36)

In [29]:
df4.loc[:, 'genres_1'] = df4.genres.map(lambda x: [x[0]]) #experiment 

In [30]:
df4.head()

,name,album,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,...,age_2011,age_2014,age_2012,age_2009,age_2013,age_2008,age_2010,age_2006,age_2017,genres_1
0,Testify,The Battle Of Los Angeles,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,...,0,0,0,0,0,0,0,0,0,[alternative_metal]
1,Guerrilla Radio,The Battle Of Los Angeles,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,...,0,0,0,0,0,0,0,0,0,[alternative_metal]
2,Calm Like a Bomb,The Battle Of Los Angeles,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,...,0,0,0,0,0,0,0,0,0,[alternative_metal]
3,Mic Check,The Battle Of Los Angeles,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,...,0,0,0,0,0,0,0,0,0,[alternative_metal]
4,Sleep Now In the Fire,The Battle Of Los Angeles,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,...,0,0,0,0,0,0,0,0,0,[alternative_metal]


In [31]:
df4.shape

(558575, 37)

In [32]:
df4.drop(columns = ['genres'], inplace = True)

In [33]:
df4.shape

(558575, 36)

In [34]:
df4['genres_1'].values[0]

['alternative_metal']

In [35]:
#creating vectorized genre features based on frequency of genre. More frequent genre = more weight
tf_idf = TfidfVectorizer()
tfidf_matrix =  tf_idf.fit_transform(df4['genres_1'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tf_idf.get_feature_names()]
genre_df.reset_index(drop = True, inplace=True)

In [36]:
genre_df.shape

(558575, 3864)

In [37]:
float_cols

array(['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'year'], dtype=object)

In [38]:
float_cols = np.delete(float_cols, 10)

In [39]:
#feature scailing for float columns in df3
floats = df4[float_cols].reset_index(drop=True)
scaler = MinMaxScaler()
scaled_float_df = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns)

In [40]:
scaled_float_df.shape

(558575, 10)

In [41]:
df4.shape

(558575, 36)

In [42]:
genre_df.shape

(558575, 3864)

In [43]:
dropper = ['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature']

In [44]:
df4.drop(columns = dropper, inplace = True)

In [45]:
df5 = pd.concat([df4, genre_df, scaled_float_df], axis = 1)

In [46]:
df5.shape

(849620, 3900)

In [44]:
df5.columns

Index(['name', 'album', 'explicit', 'key', 'mode', 'year', 'artists_final',
       'age_1996-2000', 'age_1991-1995', 'age_2018',
       ...
       'danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object', length=3900)

In [45]:
df5.isnull().sum()

name                291045
album               291045
explicit            291045
key                 291045
mode                291045
                     ...  
instrumentalness    291045
liveness            291045
valence             291045
tempo               291045
time_signature      291045
Length: 3900, dtype: int64

In [ ]:
df5[df5['name'] == np.nan]

In [47]:
df5.dropna(subset = ['name'], inplace = True)

In [48]:
df5.isnull().sum()

name                     0
album                    0
explicit                 0
key                      0
mode                     0
                     ...  
instrumentalness    291045
liveness            291045
valence             291045
tempo               291045
time_signature      291045
Length: 3900, dtype: int64

In [49]:
df5.shape

(558575, 3900)

In [52]:
df5.dropna(subset = ['tempo'], inplace = True)

In [54]:
df5.isnull().sum().sum()

0

In [56]:
df5.shape

(267530, 3900)

In [73]:
 df5['genre|21st_century_classical'].value_counts(normalize = True)

0.0    0.99985
1.0    0.00015
Name: genre|21st_century_classical, dtype: float64

In [70]:
index_list = df5.columns.tolist()

In [74]:
index_list;

In [78]:
thresh = df5['genres_1'].value_counts(normalize = True) > 0.000015

In [79]:
thresh.value_counts(normalize = True)

True     0.862241
False    0.137759
Name: genres_1, dtype: float64

In [104]:
def drop_nan_col( df, threshold): 
    #threshold = threshold
    #df = df
    for i in df.columns:
        x = (df[i]==1.0).sum()
        y = (df[i]==0.0).sum()
        if (x/(x+y)) > threshold:
            df = df.drop(i, axis=1)

In [86]:
 example = (df5['genre|21st_century_classical'] == 0.0).sum()

In [87]:
example1 = (df5['genre|21st_century_classical'] == 1.0).sum()

In [88]:
example

267490

In [89]:
example1

40

In [90]:
example1/(example + example1)

0.00014951594213733038

In [91]:
genre_df.head()

,genre|21st_century_classical,genre|48g,genre|5th_wave_emo,genre|_brasileira,genre|_hip_hop,genre|a_cappella,genre|aarhus_indie,genre|aberdeen_indie,genre|abstract,genre|abstract_beats,...,genre|zenonesque,genre|zeuhl,genre|zim_urban_groove,genre|zimdancehall,genre|zolo,genre|zouglou,genre|zouk,genre|zouk_riddim,genre|zurich_indie,genre|zydeco
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
genre_df['genre|estonian_jazz']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
558570    0.0
558571    0.0
558572    0.0
558573    0.0
558574    0.0
Name: genre|estonian_jazz, Length: 558575, dtype: float64

In [98]:
genre_df['genre|21st_century_classical']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
558570    0.0
558571    0.0
558572    0.0
558573    0.0
558574    0.0
Name: genre|21st_century_classical, Length: 558575, dtype: float64

In [99]:
genre_df.shape

(558575, 3864)

In [106]:
df5.shape

(267530, 3900)

In [107]:
df5.head()

,name,album,explicit,key,mode,year,artists_final,age_1996-2000,age_1991-1995,age_2018,...,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,Testify,The Battle Of Los Angeles,0.0,7.0,1.0,1999.0,[Rage Against The Machine],1.0,0.0,0.0,...,0.470,0.978,0.812104,0.075026,0.026205,0.000011,0.3560,0.503,0.477578,0.8
1,Guerrilla Radio,The Battle Of Los Angeles,1.0,11.0,1.0,1999.0,[Rage Against The Machine],1.0,0.0,0.0,...,0.599,0.957,0.806675,0.194014,0.012952,0.000071,0.1550,0.489,0.419956,0.8
2,Calm Like a Bomb,The Battle Of Los Angeles,0.0,7.0,1.0,1999.0,[Rage Against The Machine],1.0,0.0,0.0,...,0.315,0.970,0.811732,0.498452,0.023494,0.000002,0.1220,0.370,0.606559,0.8
3,Mic Check,The Battle Of Los Angeles,1.0,11.0,0.0,1999.0,[Rage Against The Machine],1.0,0.0,0.0,...,0.440,0.967,0.805694,0.244582,0.163655,0.000004,0.1210,0.574,0.391894,0.8
4,Sleep Now In the Fire,The Battle Of Los Angeles,0.0,2.0,1.0,1999.0,[Rage Against The Machine],1.0,0.0,0.0,...,0.426,0.929,0.792322,0.072343,0.001627,0.105000,0.0789,0.539,0.514653,0.8


In [110]:
df5.select_dtypes(include = 'O')

,name,album,artists_final,genres_1
0,Testify,The Battle Of Los Angeles,[Rage Against The Machine],[alternative_metal]
1,Guerrilla Radio,The Battle Of Los Angeles,[Rage Against The Machine],[alternative_metal]
2,Calm Like a Bomb,The Battle Of Los Angeles,[Rage Against The Machine],[alternative_metal]
3,Mic Check,The Battle Of Los Angeles,[Rage Against The Machine],[alternative_metal]
4,Sleep Now In the Fire,The Battle Of Los Angeles,[Rage Against The Machine],[alternative_metal]
...,...,...,...,...
558557,Mix Lalo Tzul: Cuando Te Vayas / Alegres Taxis...,Folklore Vol. 7. Música de Guatemala para los ...,[Marimba Maria Concepcion],[marimba_orquesta]
558558,Traicionera - En Vivo,Folklore Vol. 7. Música de Guatemala para los ...,[Marimba Maria Concepcion],[marimba_orquesta]
558559,Costa Norte - En Vivo,Folklore Vol. 7. Música de Guatemala para los ...,[Marimba Maria Concepcion],[marimba_orquesta]
558560,Las Chancletas de Nayo Capero - En Vivo,Folklore Vol. 7. Música de Guatemala para los ...,[Marimba Maria Concepcion],[marimba_orquesta]


In [111]:
df5.to_csv('spotify_genres.csv')

In [112]:
dropper2 = ['name', 'album', 'artists_final', 'genres_1']

In [113]:
df5.drop(columns = dropper2, inplace = True)

In [115]:
df5.drop(columns = ['year'], inplace = True)

In [119]:
# Fit on DTM
nn = NearestNeighbors(n_neighbors=6, algorithm= 'brute', n_jobs = -1)
nn.fit(df5)

NearestNeighbors(algorithm='brute', n_jobs=-1, n_neighbors=6)

In [120]:
# sample a doc from dtm to use as our query point 
doc_index = 236
doc = [df5.iloc[doc_index].values]

# Query Using kneighbors 
neigh_dist, neigh_index = nn.kneighbors(doc)

print(neigh_dist)
print(neigh_index)

[[0.         0.32108115 0.44185656 1.05680907 1.07796609 1.09888592]]
[[   236    240  35524 262780  35520  35519]]


In [127]:
df.iloc[262780]

id                           2zdmlXzV4uYj4kGMLtNW29
name                Introduction, Romance and Waltz
album                                Ewan Dobson II
album_id                     0Eh3IjZwngziLClkaJXfiJ
artists                             ['Ewan Dobson']
artist_ids                   68Tu5kmhurThXqZeRgsKi1
track_number                                      8
disc_number                                       1
explicit                                      False
danceability                                  0.573
energy                                        0.393
key                                               0
loudness                                    -11.884
mode                                              1
speechiness                                   0.175
acousticness                                  0.785
instrumentalness                              0.845
liveness                                       0.11
valence                                       0.253
tempo       

In [131]:
df5.iloc[236]

explicit            0.000000
key                 4.000000
mode                1.000000
age_1996-2000       1.000000
age_1991-1995       0.000000
                      ...   
instrumentalness    0.000000
liveness            0.110000
valence             0.356000
tempo               0.485169
time_signature      0.800000
Name: 257, Length: 3895, dtype: float64

In [130]:
;

''